# Домашнее задание № 6

1. Реализовать с использованием потоков и процессов скачивание файлов из интернета. 
2. Список файлов для скачивания подготовить самостоятельно (например изображений, не менее 100 изображений или других объектов). Сравнить производительность с последовательным методом. 
3. Сравнивть производительность Thread и multiprocessing решений. Попробовать подобрать оптимальное число потоков/процессов. 

    

## Подготовка данных для скачивания

In [1]:
import time
import requests
from bs4 import BeautifulSoup
import os
import shutil

list_url = ["https://www.onliner.by/"]
images = []
for url in list_url:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, features="html.parser")
    for img in soup.findAll('img'):
        images.append(img.get('src'))

cleaned_img = []
for i in range(len(images)):
    if images[i]==None or images[i].startswith('//'):
        continue
    elif images[i].endswith('jpg'):
        cleaned_img.append(images[i])

print(len(cleaned_img))#количество скачанных изображений

135


## Cкачивание изображений последовательно

In [2]:
def download_image_without_thread(download_list_url, number, dir_name):
    file_path=dir_name+'\\image'+str(number)+'.jpg'
    img=requests.get(download_list_url)
    open(file_path,"wb").write(img.content)
    
    
try: 
    os.mkdir('Images')
except FileExistsError:
    shutil.rmtree ('Images')
    os.mkdir ('Images')
    
print("Running without threads:")
start=time.time()

for i in cleaned_img:
    download_image_without_thread(i,cleaned_img.index(i),'Images')
end = time.time()

print ("На загрузку потрачено - ", end-start, ' секунд')
print ("Скачано - ", len(cleaned_img), " файлов")

Running without threads:
На загрузку потрачено -  8.147306680679321  секунд
Скачано -  135  файлов


## Использование потоков при скачивании изображений 
## из интернета с использованием ThreadPoolExecutor()

In [6]:
import concurrent.futures

try: 
    os.mkdir('ImagesThreads')
except FileExistsError:
    shutil.rmtree ('ImagesThreads')
    os.mkdir ('ImagesThreads')

    
def download_with_ThreadPoolExecutor(download_list_url, number, dir_name):
    file_path=dir_name+'\\image'+str(number)+'.jpg'
    img=requests.get(download_list_url)
    open(file_path,"wb").write(img.content)
    
print("Running threaded:")
threaded_start = time.time()

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for url in cleaned_img:
        futures.append(executor.submit(download_with_ThreadPoolExecutor, 
                                       download_list_url = url, 
                                       number = cleaned_img.index(url), 
                                       dir_name = 'ImagesThreads'))

threaded_end = time.time()

print ("На загрузку потрачено - ", threaded_end - threaded_start, ' секунд')
print ("Скачано - ", len(cleaned_img), " файлов")

Running threaded:
На загрузку потрачено -  2.2037434577941895  секунд
Скачано -  135  файлов


## Использование мультипроцессинга

In [ ]:
from multiprocessing import Pool, cpu_count

try: 
    os.mkdir('MultyProcess')
except FileExistsError:
    shutil.rmtree ('MultyProcess')
    os.mkdir('MultyProcess')

def img_downl_mult(url):
    file_path='MultyProcess\\image' + str(cleaned_img.index(url)) + '.jpg'
    img=requests.get(url)
    open(file_path,"wb").write(img.content)

if __name__ == '__main__':
    pool = Pool(15)
    start = time.time()
    results = pool.map(img_downl_mult, cleaned_img)
    pool.close()
    pool.join()
    end = time.time()
    print('Time taken in seconds -', end - start)

### Скачивание изображений с помощью мультипроцессинга запускалось через командную строку. Для это код программы был отдельно вынесен в файл HW_06_with_multyprocess.py, который так же находится в папке с шестым домашним заданием. Результаты запуска программы:

![jupyter](./download_with_multyprocess.png)

## Вывод:

Таким образом наиболее неэффективным вариантом скачивания изображения из интернета является последовательное скачивание изображений(8 с), следом идет использование мультипроцессинга(4 с), а наиболее эффективным вариантом скачивания изобрадений стал многопоточный вариант с использованием ThreadPoolExecutor().